In [19]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import pandas as pd
import numpy as np
import time

from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.advanced_activations import PReLU
from keras.optimizers import Adam, Nadam, SGD

from models_utils_fe import *

In [5]:
src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/scripts/features/'
feats_src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/data/features/uncleaned/'

X_train = pd.read_pickle('Xtrain_500bestCols.pkl')

xgb_feats = pd.read_csv(feats_src + '/the_1owl/owl_train.csv')
y_train = xgb_feats[['is_duplicate']]

In [24]:
def dense_model(ncols):
    dense_input = Input(shape = (ncols,))
    d = Dense(256, kernel_initializer = 'he_normal')(dense_input)
    d = PReLU()(d)
    d = BatchNormalization()(d)
    d = Dropout(0.4)(d)
    
    d2 = Dense(512, kernel_initializer = 'he_normal')(d)
    d2 = PReLU()(d2)
    d2 = BatchNormalization()(d2)
    d2 = Dropout(0.4)(d2)
    
    d2 = Dense(512, kernel_initializer = 'he_normal')(d2)
    d2 = PReLU()(d2)
    d2 = BatchNormalization()(d2)
    d2 = Dropout(0.4)(d2)
    
    d3 = Dense(512, kernel_initializer = 'he_normal')(d2)
    d3 = PReLU()(d3)
    d3 = BatchNormalization()(d3)
    d3 = Dropout(0.4)(d3)
    preds = Dense(1, activation='sigmoid')(d3)
    
    model = Model(inputs=[dense_input], outputs=preds)
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr = 5e-5), metrics=['acc'])
    return model

def train_mlp():
    
    t = time.time()
    X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, stratify = y_train,
                                            test_size = 0.2, random_state = 111)
    ncols = X_train.shape[1]
    model = dense_model(ncols)

    early_stopping = EarlyStopping(monitor='val_loss', patience = 5)
    hist = model.fit(X_tr.values, y_tr.values, validation_data=(X_val.values, y_val.values), 
                     epochs=200, batch_size=128,
                     verbose=1, shuffle=True, callbacks=[early_stopping])
    val_pred = model.predict(X_val)
    
    score = log_loss(y_val, val_pred)
    print('Final score:', score, '\n', 'Time it took to train and predict:', time.time() - t)
    return model

In [25]:
mlp = train_mlp()

Train on 323432 samples, validate on 80858 samples
Epoch 1/200
323432/323432 [==============================] - 33s - loss: 0.6603 - acc: 0.6308 - val_loss: 0.6586 - val_acc: 0.6308
Epoch 2/200
 49024/323432 [===>..........................] - ETA: 21s - loss: 0.6582 - acc: 0.6314

KeyboardInterrupt: 